## Data collection toolkit for monitoring "peer effects"

This cell sets up the notebook to import numpy, datascience, seaborn, pandas, matplotlib etc.

In [1]:
# Run this cell to set up the notebook.

# These lines import the Numpy, Datascience, pandas modules.
import numpy as np
import pandas as pd
import seaborn as sns
from datascience import *
import datetime as dt
import random
import matplotlib
import matplotlib.pyplot as plt

# Importing plotting libraries and styles
%matplotlib inline
plt.style.use('fivethirtyeight')

# For Pandas to ignore FutureWarning displays
import warnings
warnings.simplefilter('ignore', FutureWarning)

Defining the class `Person`

In [2]:
class Person:
    def __init__(self, index, startTime):
        self.index = index
        self.startTime = startTime
        self.breakStart = []
        self.breakEnd = []
    
    def addBreak(self, time):
        self.breakStart.append(time)
        return "Break started for person " + str(self.index) + " at " + str(time)
    
    def endBreak(self, time):
        if (len(self.breakStart)-1)!=len(self.breakEnd):
            return "Please start a break to stop it."
        else:
            self.breakEnd.append(time)
            return "Break ended for person " + str(self.index) + " at " + str(time)
        
    def numBreaks(self):
        return len(self.breakEnd)

Defining the class `Table`

In [3]:
class Table:
    listTables = []
    
    def __init__(self, name, libraryName = '', people = 0):
        self.name = name
        self.startTime = dt.datetime.now()
        self.library = libraryName
        self.persons = []
        for i in range(1, people+1):
            self.addPerson()
        Table.listTables.append(self)
        print("Table with name " + str(name) + " and " + str(people) + " people" + " created!")
        print("Time: " + str(self.startTime))
        
    def addPerson(self):
        number = len(self.persons)+1
        self.persons.append(Person(number, dt.datetime.now()))
        return "Person number " + str(number) + " added."
    
    def addBreak(self, name):
        if name > len(self.persons):
            return "Person " + str(name) + " not in list of people."
        person = self.persons[name-1]
        time = dt.datetime.now()
        return person.addBreak(time)
        
    def endBreak(self, name):
        if name > len(self.persons):
            return "Person " + str(name) + " not in list of people."
        time = dt.datetime.now()
        person = self.persons[name-1]
        return person.endBreak(time)
        #return "Break ended for person " + str(name) + " at " + str(time)
    
    def getNumBreaks(self, name):
        person = self.persons[name-1]
        return "Person " + str(name) + " has taken " + str(person.numBreaks()) + " breaks."

Creating a wrapper class `ListTables`

Todo:
* Easy way to process and input time
    * Find convenient format for everyone

**Features:**
* Person departure
* Person arrival
* Person break begins
* Person checking phone

### Demo

In [4]:
# Creates a new table named 1 in Moffit Library
A = Table("A", "Moffit", 4)

# Adds person 5 to table 1
A.addPerson()

Table with name A and 4 people created!
Time: 2019-02-18 23:30:25.810681


'Person number 5 added.'

In [5]:
# Person 5 on Table A leaves for a break
A.addBreak(5)

'Break started for person 5 at 2019-02-18 23:30:26.460270'

In [6]:
B = Table("B", "Stacks", 3)
C = Table("C", "Moffit", 10)

# Person 2 and 3 on Table B leave together for a break
B.addBreak(2)
B.addBreak(3)

Table with name B and 3 people created!
Time: 2019-02-18 23:30:27.407470
Table with name C and 10 people created!
Time: 2019-02-18 23:30:27.407734


'Break started for person 3 at 2019-02-18 23:30:27.407883'

In [7]:
# Person 5 on Table A comes back
A.endBreak(5)

# Person 2 and 3 on Table B come back
B.endBreak(2)
B.endBreak(3)

'Break ended for person 3 at 2019-02-18 23:30:28.547209'

In [8]:
# A5 leaves for break and comes back again
A.addBreak(5)
A.endBreak(5)

'Break ended for person 5 at 2019-02-18 23:30:29.107632'

In [9]:
# Getting number of breaks taken by A5
A.getNumBreaks(5)

'Person 5 has taken 2 breaks.'

Todo:
* Ask Max and others for format/appropriate features for an csv, Excel file
* Write code to convert single library session into the appropriate csv file

Other thoughts/functionalities that should be added?

In [ ]:
# This cell converts all the break information into a usable CSV file

**Format:** Person, Table, Library, Arrival Time, Departure Time, Num Breaks, Break Start Times (list), Break End Times (list), Neighbours, Known